GLIE: Greedy in the Limit with Infinite Exploration

In [1]:
from rl.monte_carlo import *

from rl.function_approx import Tabular
from rl.distribution import Choose
from rl.chapter3.simple_inventory_mdp_cap import InventoryState
from rl.chapter10.prediction_utils import *

from rl.chapter3.simple_inventory_mdp_cap import SimpleInventoryMDPCap
from rl.dynamic_programming import value_iteration_result

from rl.distribution import Constant
from rl.dynamic_programming import V
import itertools
import rl.iterate as iterate
from rl.markov_decision_process import FiniteMarkovDecisionProcess
from rl.policy import FiniteDeterministicPolicy

In [2]:
capacity: int = 2
poisson_lambda: float = 1.0
holding_cost: float = 1.0
stockout_cost: float = 10.0
gamma: float = 0.9
si_mdp: SimpleInventoryMDPCap = SimpleInventoryMDPCap(
    capacity=capacity,
    poisson_lambda=poisson_lambda,
    holding_cost=holding_cost,
    stockout_cost=stockout_cost
)

In [3]:

true_opt_vf, true_opt_policy = value_iteration_result(si_mdp, gamma=gamma)
print("True Optimal Value Function")
pprint(true_opt_vf)
print("True Optimal Policy")
print(true_opt_policy)

True Optimal Value Function
{NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -34.894855194671294,
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.66095964467877,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -27.99189950444479,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -28.66095964467877,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -28.99189950444479,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -29.991899504444792}
True Optimal Policy
For State InventoryState(on_hand=0, on_order=0): Do Action 1
For State InventoryState(on_hand=0, on_order=1): Do Action 1
For State InventoryState(on_hand=0, on_order=2): Do Action 0
For State InventoryState(on_hand=1, on_order=0): Do Action 1
For State InventoryState(on_hand=1, on_order=1): Do Action 0
For State InventoryState(on_hand=2, on_order=0): Do Action 0



In [8]:
episode_length_tolerance: float = 1e-5
epsilon_as_func_of_episodes: Callable[[int], float] = lambda k: k ** -0.5

initial_learning_rate: float = 0.1
half_life: float = 10000.0
exponent: float = 1.0

# Uniform sampling across state space:
initial_qvf_dict = {
    (s, a): 0. for s in si_mdp.non_terminal_states for a in si_mdp.actions(s)
}
learning_rate_func: Callable[[int], float] = learning_rate_schedule(
    initial_learning_rate=initial_learning_rate,
    half_life=half_life,
    exponent=exponent
)
qvfs = glie_mc_control(
    mdp=si_mdp,
    states=Choose(si_mdp.non_terminal_states),
    approx_0=Tabular(
        values_map=initial_qvf_dict,
        count_to_weight_func=learning_rate_func
    ),
    γ=gamma,
    ϵ_as_func_of_episodes=epsilon_as_func_of_episodes,
    episode_length_tolerance=episode_length_tolerance
)

In [9]:
QTabular = Tabular[Tuple[NonTerminal[S],A]]

In [10]:
# Tabular Monte-Carlo Control

num_episodes = 10000
final_qvf: QTabular[InventoryState, int] = \
    iterate.last(itertools.islice(qvfs, num_episodes))

def tabular_get_vf_and_policy_from_qvf(
    mdp: FiniteMarkovDecisionProcess[S, A],
    qvf: QTabular[S, A]
) -> Tuple[V[S], FiniteDeterministicPolicy[S, A]]:
    opt_vf: V[S] = {
        s: max(qvf((s, a)) for a in mdp.actions(s))
        for s in mdp.non_terminal_states
    }
    opt_policy: FiniteDeterministicPolicy[S, A] = \
        FiniteDeterministicPolicy({
            s.state: qvf.argmax((s, a) for a in mdp.actions(s))[1]
            for s in mdp.non_terminal_states
        })
    return opt_vf, opt_policy
opt_vf, opt_policy = tabular_get_vf_and_policy_from_qvf(
    mdp=si_mdp,
    qvf=final_qvf
)
print("GLIE MC Optimal Value Function with {num_episodes:d} episodes")
pprint(opt_vf)
print("GLIE MC Optimal Policy with {num_episodes:d} episodes")
print(opt_policy)

GLIE MC Optimal Value Function with {num_episodes:d} episodes
{NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -34.96476984562754,
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.53625227302578,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -28.586970994259534,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -28.581369243292652,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -28.825334895696155,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -30.1696604747163}
GLIE MC Optimal Policy with {num_episodes:d} episodes
For State InventoryState(on_hand=0, on_order=0): Do Action 1
For State InventoryState(on_hand=0, on_order=1): Do Action 1
For State InventoryState(on_hand=0, on_order=2): Do Action 0
For State InventoryState(on_hand=1, on_order=0): Do Action 1
For State InventoryState(on_hand=1, on_order=1): Do Action 0
For State InventoryState(on_hand=2, on_order=0): Do Action 0



In [11]:
# Monte-Carlo Control with Generic Function Approximation

num_episodes = 10000
final_qvf: QValueFunctionApprox[InventoryState, int] = \
    iterate.last(itertools.islice(qvfs, num_episodes))

def get_vf_and_policy_from_qvf(
    mdp: FiniteMarkovDecisionProcess[S, A],
    qvf: QValueFunctionApprox[S, A]
) -> Tuple[V[S], FiniteDeterministicPolicy[S, A]]:
    opt_vf: V[S] = {
        s: max(qvf((s, a)) for a in mdp.actions(s))
        for s in mdp.non_terminal_states
    }
    opt_policy: FiniteDeterministicPolicy[S, A] = \
        FiniteDeterministicPolicy({
            s.state: qvf.argmax((s, a) for a in mdp.actions(s))[1]
            for s in mdp.non_terminal_states
        })
    return opt_vf, opt_policy
opt_vf, opt_policy = get_vf_and_policy_from_qvf(
    mdp=si_mdp,
    qvf=final_qvf
)
print("GLIE MC Optimal Value Function with {num_episodes:d} episodes")
pprint(opt_vf)
print("GLIE MC Optimal Policy with {num_episodes:d} episodes")
print(opt_policy)

GLIE MC Optimal Value Function with {num_episodes:d} episodes
{NonTerminal(state=InventoryState(on_hand=0, on_order=0)): -35.00813385042989,
 NonTerminal(state=InventoryState(on_hand=0, on_order=1)): -27.765313656007862,
 NonTerminal(state=InventoryState(on_hand=0, on_order=2)): -28.502592586157864,
 NonTerminal(state=InventoryState(on_hand=1, on_order=0)): -28.78768232944734,
 NonTerminal(state=InventoryState(on_hand=1, on_order=1)): -29.13917516743337,
 NonTerminal(state=InventoryState(on_hand=2, on_order=0)): -30.15988517065721}
GLIE MC Optimal Policy with {num_episodes:d} episodes
For State InventoryState(on_hand=0, on_order=0): Do Action 1
For State InventoryState(on_hand=0, on_order=1): Do Action 1
For State InventoryState(on_hand=0, on_order=2): Do Action 0
For State InventoryState(on_hand=1, on_order=0): Do Action 1
For State InventoryState(on_hand=1, on_order=1): Do Action 0
For State InventoryState(on_hand=2, on_order=0): Do Action 0



### Copy the above as the generic "Function Approximation" case. Replace this version with the Tabular setting. Might need to do a reclassification of Tabuluar as was done for QValueFunctionApprox[S,A]

## Implement Asset_alloc_discrete test

In [14]:
from rl.chapter7.asset_alloc_discrete import *
from pprint import pprint

In [15]:
steps: int = 4
μ: float = 0.13
σ: float = 0.2
r: float = 0.07
a: float = 1.0
init_wealth: float = 1.0
init_wealth_stdev: float = 0.1

excess: float = μ - r
var: float = σ * σ
base_alloc: float = excess / (a * var)

risky_ret: Sequence[Gaussian] = [Gaussian(μ=μ, σ=σ) for _ in range(steps)]
riskless_ret: Sequence[float] = [r for _ in range(steps)]
utility_function: Callable[[float], float] = lambda x: - np.exp(-a * x) / a
alloc_choices: Sequence[float] = np.linspace(
    2 / 3 * base_alloc,
    4 / 3 * base_alloc,
    11
)
feature_funcs: Sequence[Callable[[Tuple[float, float]], float]] = \
    [
        lambda _: 1.,
        lambda w_x: w_x[0],
        lambda w_x: w_x[1],
        lambda w_x: w_x[1] * w_x[1]
    ]
dnn: DNNSpec = DNNSpec(
    neurons=[],
    bias=False,
    hidden_activation=lambda x: x,
    hidden_activation_deriv=lambda y: np.ones_like(y),
    output_activation=lambda x: - np.sign(a) * np.exp(-x),
    output_activation_deriv=lambda y: -y
)
init_wealth_distr: Gaussian = Gaussian(μ=init_wealth, σ=init_wealth_stdev)

aad: AssetAllocDiscrete = AssetAllocDiscrete(
    risky_return_distributions=risky_ret,
    riskless_returns=riskless_ret,
    utility_func=utility_function,
    risky_alloc_choices=alloc_choices,
    feature_functions=feature_funcs,
    dnn_spec=dnn,
    initial_wealth_distribution=init_wealth_distr
)

In [16]:
aa_mdp = aad.get_mdp(2)

In [17]:
#Optimal Value Function via ADP

vf_ff: Sequence[Callable[[NonTerminal[float]], float]] = [lambda _: 1., lambda w: w.state]
it_vf: Iterator[Tuple[DNNApprox[NonTerminal[float]], DeterministicPolicy[float, float]]] = \
    aad.backward_induction_vf_and_pi(vf_ff)

print("Backward Induction: VF And Policy")
print("---------------------------------")
print()
for t, (v, p) in enumerate(it_vf):
    print(f"Time {t:d}")
    print()
    opt_alloc: float = p.action_for(init_wealth)
    val: float = v(NonTerminal(init_wealth))
    print(f"Opt Risky Allocation = {opt_alloc:.2f}, Opt Val = {val:.3f}")
    print("Weights")
    for w in v.weights:
        print(w.weights)
    print()

Backward Induction: VF And Policy
---------------------------------

Time 0

Opt Risky Allocation = 1.70, Opt Val = -0.213
Weights
[[0.23191951 1.31664746]]

Time 1

Opt Risky Allocation = 1.00, Opt Val = -0.246
Weights
[[0.1729801  1.23037967]]

Time 2

Opt Risky Allocation = 1.70, Opt Val = -0.283
Weights
[[0.11618034 1.1474794 ]]

Time 3

Opt Risky Allocation = 1.10, Opt Val = -0.323
Weights
[[0.05786796 1.07137133]]



## This is not the right way to do this:

#### Need a finite MDP to run algorithm as currently specified. 

In [21]:
# Optimal Value Function via Monte-Carlo Control


# Uniform sampling across state space:
initial_qvf_dict = {
    (s, a): 0. for s in aa_mdp.non_terminal_states for a in aa_mdp.actions(s)
}
learning_rate_func: Callable[[int], float] = learning_rate_schedule(
    initial_learning_rate=initial_learning_rate,
    half_life=half_life,
    exponent=exponent
)
qvfs = glie_mc_control(
    mdp=aa_mdp,
    states=Choose(aa_mdp.non_terminal_states),
    approx_0=Tabular(
        values_map=initial_qvf_dict,
        count_to_weight_func=learning_rate_func
    ),
    γ=gamma,
    ϵ_as_func_of_episodes=epsilon_as_func_of_episodes,
    episode_length_tolerance=episode_length_tolerance
)



AttributeError: 'AssetAllocMDP' object has no attribute 'non_terminal_states'

In [ ]:
num_episodes = 10000
final_qvf: QValueFunctionApprox[InventoryState, int] = \
    iterate.last(itertools.islice(qvfs, num_episodes))

opt_vf, opt_policy = get_vf_and_policy_from_qvf(
    mdp=si_mdp,
    qvf=final_qvf
)
print("GLIE MC Optimal Value Function with {num_episodes:d} episodes")
pprint(opt_vf)
print("GLIE MC Optimal Policy with {num_episodes:d} episodes")
print(opt_policy)